In [ ]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

# -----------------------------------------------------------------------------
# Script Description:
# This script loads the original SNF interpolation grid database (grid_database.parq),
# reduces the grid points according to a new experiment setting (exp_folder_name),
# and saves the reduced dataset as a new Parquet file.
#
# Example:
#   exp_folder_name = "1412"
#   => enrich_factor = 1, sp_factor = 4, bp_factor = 1, cool_factor = 2
#
# The reduced dataset is written to: grid_database_1412.parq
# -----------------------------------------------------------------------------

def reduce_grid_database(input_file: Path, exp_folder_name: str, output_file: Path) -> None:
    """
    Reduce the interpolation grid size according to the given experiment factors.

    Parameters
    ----------
    input_file : Path
        Path to the original grid_database.parq file.
    exp_folder_name : str
        Four-digit string encoding reduction factors for [enrich, sp, burnup, cool].
    output_file : Path
        Path to save the reduced grid database in Parquet format.
    """

    # -------------------------------------------------------------------------
    # Step 1: Parse reduction factors from exp_folder_name
    # -------------------------------------------------------------------------
    enrich_factor = int(exp_folder_name[0])
    sp_factor = int(exp_folder_name[1])
    bp_factor = int(exp_folder_name[2])
    cool_factor = int(exp_folder_name[3])

    # -------------------------------------------------------------------------
    # Step 2: Define the original full interpolation spaces
    # -------------------------------------------------------------------------
    enrich_space = np.arange(1.5, 6.1, 0.5)[0::enrich_factor]
    sp_space = np.arange(5, 46, 5)[0::sp_factor]
    burnup_space = np.arange(5000, 74100, 3000)[0::bp_factor]
    cool_space_raw = np.logspace(-5.75, 6.215, 150, base=math.e)
    cool_space = cool_space_raw[1::cool_factor]

    # -------------------------------------------------------------------------
    # Step 3: Load the original dataset
    # -------------------------------------------------------------------------
    df = pd.read_parquet(input_file)

    # -------------------------------------------------------------------------
    # Step 4: Filter the dataset according to reduced interpolation spaces
    # -------------------------------------------------------------------------
    df_reduced = df[
        df["Enrich"].isin(enrich_space)
        & df["SP"].isin(sp_space)
        & df["Burnup"].isin(burnup_space)
        & df["Cool"].isin(cool_space)
    ].reset_index(drop=True)

    # -------------------------------------------------------------------------
    # Step 5: Save the reduced dataset
    # -------------------------------------------------------------------------
    df_reduced.to_parquet(output_file, index=False)
    print(
        f"Reduced grid database saved to {output_file} "
        f"with {df_reduced.shape[0]} rows (from {df.shape[0]} original rows)."
    )


if __name__ == "__main__":
    # Define paths
    project_root = Path.cwd()
    input_data_file = project_root / "data" / "grid_database.parq"
    exp_folder_name = "1412"
    output_data_file = project_root / "data" / f"grid_database_{exp_folder_name}.parq"

    # Run the reduction process
    reduce_grid_database(input_data_file, exp_folder_name, output_data_file)
